In [1]:
import numpy as np
import pandas as pd
import sys
import os
import torch
sys.path.insert(0, os.path.abspath("src/"))

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
current_directory = os.path.abspath('.')  # Get the absolute path of the current directory
parent_directory = os.path.dirname(current_directory)  # Get the directory name one level up
basepath = parent_directory
datapath = os.path.join(basepath, "data")
batch_size = 100
seed = 31415

In [3]:
print(datapath)

D:\JKU\W2023\PW\Amumo\data


In [4]:
from amumo.data import *

# Load Data
dataset_cloome = CLOOMDataset_Dataset(datapath,seed,batch_size)
cloome_images, cloome_molecules, cloome_dataset_name = dataset_cloome.get_data()
cloome_dataset_name

'CLOOMDataset_size-100'

In [5]:
# from PIL import Image
# for i in range(len(cloome_images.data)):
#     # Convert each NumPy array representing an image to a PIL Image and overwrite the array element
#     cloome_images.data[i] = Image.fromarray(cloome_images.data[i])

In [6]:
type(cloome_images.data[0])

PIL.Image.Image

In [7]:
from amumo.utils import *
from amumo.model import *
molecule_features = os.path.join(datapath, "all_molecule_cellpainting_features.pkl")
image_features = os.path.join(datapath, "subset_image_cellpainting_features.pkl")

# molecule features
mol_features_torch = torch.load(molecule_features, map_location=device)
mol_features = mol_features_torch["mol_features"]
mol_ids = mol_features_torch["mol_ids"]

# microscopy features
img_features_torch = torch.load(image_features, map_location=device)
img_features = img_features_torch["img_features"]
img_ids = img_features_torch["img_ids"]

# extract subsets of features
img_feature_idcs = [np.where(np.array(img_features_torch["img_ids"])==i)[0][0] for i in dataset_cloome.dataset["SAMPLE_KEY_img"].values]
mol_feature_idcs = [np.where(np.array(mol_features_torch["mol_ids"])==i)[0][0] for i in dataset_cloome.dataset["SAMPLE_KEY_mol"].values]

mol_features_sample = mol_features_torch['mol_features'][mol_feature_idcs]
mol_features_sample = l2_norm(mol_features_sample)

img_features_sample = img_features_torch['img_features'][img_feature_idcs]
img_features_sample = l2_norm(img_features_sample)

# cache features
cloome_model = PrecalculatedModel('seed-%i'%seed, cloome_dataset_name, img_features_sample, mol_features_sample)

In [8]:
from amumo.widgets import * 

cloome_widget = CLIPExplorerWidget(cloome_dataset_name, cloome_images, cloome_molecules, models=[cloome_model])
cloome_widget.hover_widget.width = 200
cloome_widget

C:\Users\husse\anaconda3\envs\amumo_env\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\husse\anaconda3\envs\amumo_env\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\husse\anaconda3\envs\amumo_env\lib\site-packages\umap\distances.py:1086: 

found cached embeddings for CLOOMDataset_size-100_precalculated_seed-31415


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('precalcul…